In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_state_city
from qiskit.quantum_info import random_statevector, Statevector

# Use Aer's qasm_simulator
simulator = AerSimulator()

In [2]:
#setting up the desctructive swap circuit

cr = QuantumCircuit(2, 2)

cr.cx(0,1)
cr.h(0)
cr.barrier(range(2))
cr.measure(range(2), range(2))

cr.draw()

┌───┐ ░ ┌─┐   
q_0: ──■──┤ H ├─░─┤M├───
     ┌─┴─┐└───┘ ░ └╥┘┌─┐
q_1: ┤ X ├──────░──╫─┤M├
     └───┘      ░  ║ └╥┘
c: 2/══════════════╩══╩═
                   0  1

In [9]:
#make a circuit that generates a "random" input state (do same thing to top and bottom state)

initial_state = QuantumCircuit(4)

initial_state.h([0, 2])
initial_state.s([1, 3])
initial_state.cx(0, 1)
initial_state.cx(2, 3)

initial_state.draw()

┌───┐     
q_0: ┤ H ├──■──
     ├───┤┌─┴─┐
q_1: ┤ S ├┤ X ├
     ├───┤└───┘
q_2: ┤ H ├──■──
     ├───┤┌─┴─┐
q_3: ┤ S ├┤ X ├
     └───┘└───┘

In [12]:
#alternate option where we have multiple 

cr = QuantumCircuit(4, 2)

cr.cx(0,2)
cr.cx(1,3)
#cr.h(0)
#cr.h(1)
cr.barrier(range(4))
#cr.measure(range(4), range(4))
cr.measure([2, 3], range(2))

cr.draw()

░       
q_0: ──■────────░───────
       │        ░       
q_1: ──┼────■───░───────
     ┌─┴─┐  │   ░ ┌─┐   
q_2: ┤ X ├──┼───░─┤M├───
     └───┘┌─┴─┐ ░ └╥┘┌─┐
q_3: ─────┤ X ├─░──╫─┤M├
          └───┘ ░  ║ └╥┘
c: 2/══════════════╩══╩═
                   0  1

In [13]:
#basic cirucit compilation

#compose the initial state with the measurement circuit
full_circuit = cr.compose(initial_state, range(4), front=True)

compiled = transpile(full_circuit, simulator)

total_shots = 1024

job = simulator.run(compiled, shots=total_shots)

result = job.result()

counts = result.get_counts(compiled)
print("\nTotal counts are:",counts)


Total counts are: {'11': 515, '00': 509}


In [14]:
#do the post processing, formula for overlap is 1-(2p/N) where p=# 11 measurements N = total_shots

if '00' in counts:
    p = counts['00']
else:
    p = 0

overlap = p/total_shots

print(overlap)


0.4970703125


In [11]:
#lets look at post processing for the higher dimensional system

fails = {'0101', '0111', '1010', '1011', '1101', '1110'}
p = 0

for i in fails:
    if i in counts:
        p += counts[i]

overlap = 1-2*(p/total_shots)


print(p)
print(overlap)

0
1.0
